In [5]:
from omegaconf import OmegaConf
import os

In [6]:
def load_data_paths_from_config_to_notebooks(path_name):
    notebooks_dir = os.getcwd()
    base_dir = os.path.abspath(os.path.join(notebooks_dir, ".."))
    config_path = os.path.join(base_dir, "config.yaml")
    config = OmegaConf.load(config_path)
    paths = config[path_name]
    dos_df_path = os.path.join(base_dir,paths.dos_df).replace("\\", "/")
    fuzzy_df_path = os.path.join(base_dir, paths.fuzzy_df).replace("\\", "/")
    attack_free_df_path = os.path.join(base_dir, paths.attack_free_df).replace("\\", "/")
    return dos_df_path,fuzzy_df_path,attack_free_df_path